# Famous CNN Models
This project is aimed at implementing some of the most popular CNN models in keras

### Index
- [LeNet-5](#lenet)
- [AlexNet](#alexnet)
- [VGG-16](#vgg16)

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

## Helper functions

<a id='lenet'></a>
## LeNet-5

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
# depricated
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True, reshape = False);

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
X_train = mnist.train.images
y_train = mnist.train.labels

X_test = mnist.test.images
y_test = mnist.test.labels

In [10]:
def createLeNet():
    model = Sequential()

    # First convolutional layer with filter = (5*5) and strides = 1
    model.add(Convolution2D(filters=6, kernel_size=(5, 5), input_shape=(28, 28, 1), strides=1))

    # Average pooling is applied.
    model.add(AveragePooling2D(pool_size=(2, 2), strides=1))

    # Second convolutional layer is applied with the same kernel size and the same strides.
    model.add(Convolution2D(filters=6, kernel_size=(5, 5), strides=1))

    # A final average pooling layer is applied before using Fully connected layers but with strides=2
    model.add(AveragePooling2D(pool_size=(2, 2), strides=2))

    # We then use fully connected layers

    model.add(Flatten())
    model.add(Dense(units=120))
    model.add(Dense(units=84))
    model.add(Dense(units=10, activation = 'sigmoid'))

    model.compile(optimizer = 'SGD', loss = 'mean_squared_error', metrics = ['accuracy'])
    print(model.summary())
    
    return model

In [ ]:
ReNet = createLeNet()

In [6]:
history = ReNet.fit(X_train, y_train, batch_size = 10, epochs = 15)

Epoch 1/15
55000/55000 [==============================] - 14s 262us/step - loss: 0.0470 - acc: 0.7102
Epoch 2/15
55000/55000 [==============================] - 13s 239us/step - loss: 0.0253 - acc: 0.8649
Epoch 3/15
55000/55000 [==============================] - 13s 239us/step - loss: 0.0230 - acc: 0.8779
Epoch 4/15
55000/55000 [==============================] - 13s 239us/step - loss: 0.0219 - acc: 0.8845
Epoch 5/15
55000/55000 [==============================] - 13s 240us/step - loss: 0.0213 - acc: 0.8887
Epoch 6/15
55000/55000 [==============================] - 13s 241us/step - loss: 0.0208 - acc: 0.8899
Epoch 7/15
55000/55000 [==============================] - 13s 243us/step - loss: 0.0205 - acc: 0.8908
Epoch 8/15
55000/55000 [==============================] - 13s 243us/step - loss: 0.0203 - acc: 0.8931
Epoch 9/15
55000/55000 [==============================] - 13s 242us/step - loss: 0.0200 - acc: 0.8938
Epoch 10/15
55000/55000 [==============================] - 13s 245us/step - loss: 

In [9]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.017409637355990708
Test accuracy: 0.9099
